In [2]:
import six.moves.urllib.request as urlreq

import dash
import dash_bio as dashbio
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash_bio_utils import pdb_parser as parser
from dash.dependencies import Input, Output
import json
import tempfile

import copy
import pandas as pd

try:
    import mutagenesis_visualization as mut
except ModuleNotFoundError:  # This step is only for when I run the notebooks locally
    import sys
    sys.path.append('../../../')
    import mutagenesis_visualization as mut

# local modules
try:
    import mutagenesis_visualization.main.scripts.code_kwargs as code_kwargs
    import mutagenesis_visualization.main.scripts.code_utils as code_utils
except ModuleNotFoundError:
    import import_notebook
    import code_kwargs
    import code_utils

In [3]:
hras_RBD = mut.hras_RBD()

In [103]:
pdb = '../../data/5p21.pdb'
xx = dashboard_3d_protein(hras_RBD, pdb)

In [106]:
xx.show()

AttributeError: 'Molecule3dViewer' object has no attribute 'show'

In [123]:
def dashboard_3d_protein(self, pdb, chain='A', position_correction=0, **kwargs):
    '''
    Docstring
    '''

    # update kwargs
    temp_kwargs = copy.deepcopy(code_kwargs.kwargs())
    temp_kwargs.update(kwargs)

    # Load data from pdb file
    model_data = _parse_pdb(pdb)

    # Run dashhoard
    _run_dashboard_3d_protein(
        self, model_data, temp_kwargs, position_correction, chain
    )


def _run_dashboard_3d_protein(
    self, model_data, temp_kwargs, position_correction, chain
):
    '''
    Create dashboard for main function.
    '''

    # Open app
    app = dash.Dash('__main__', external_stylesheets=[dbc.themes.BOOTSTRAP])

    # App layout
    app.layout = html.Div([
        dbc.Row(
            dbc.Col(
                html.H3("Site-saturation mutagenesis of H-Ras"),  # title
                width={'size': 12},
            ),
        ),
        dbc.Row(
            dbc.Col(
                dcc.Graph(  # heatmap
                    id="heatmap",
                    figure={},
                ),
                width=12,
            )
        ),
        dbc.Row([
            dbc.Col(
                html.Div("Choose how to filter the data"),
                width={'size': 4, 'offset': 2}
            ),
            dbc.Col(
                dcc.Dropdown(
                    id='dropdown',
                    placeholder='choose amino acid',
                    clearable=False,
                    value='mean',
                    options=[
                        {'label': 'Mean', 'value': 'mean'},
                        {'label': 'Alanine', 'value': 'A'},
                        {'label': 'Arginine', 'value': 'R'},
                        {'label': 'Asparagine', 'value': 'N'},
                        {'label': 'Aspartic acid ', 'value': 'D'},
                        {'label': 'Cysteine', 'value': 'C'},
                        {'label': 'Glutamine', 'value': 'Q'},
                        {'label': 'Glutamic acid ', 'value': 'E'},
                        {'label': 'Glycine', 'value': 'G'},
                        {'label': 'Histidine', 'value': 'H'},
                        {'label': 'Isoleucine', 'value': 'I'},
                        {'label': 'Leucine', 'value': 'L'},
                        {'label': 'Lysine', 'value': 'K'},
                        {'label': 'Methionine', 'value': 'M'},
                        {'label': 'Phenylalanine', 'value': 'F'},
                        {'label': 'Proline', 'value': 'P'},
                        {'label': 'Serine', 'value': 'S'},
                        {'label': 'Threonine', 'value': 'T'},
                        {'label': 'Tryptophan', 'value': 'W'},
                        {'label': 'Tyrosine', 'value': 'Y'},
                        {'label': 'Valine', 'value': 'V'},
                        #{'label': 'Stop codon', 'value': '*'},
                    ]
                ),
                width=4,
            ),
        ]),
        dbc.Row([
            dbc.Col(
                id='moleculeviewer',
                children={},
                width={"size": 6, "order": 1},
            ),
            dbc.Col(
                dcc.Graph(
                    id="mean",
                    figure={},
                ),
                width={"size": 6, "order": 2},
            ),
        ],
                no_gutters=True),
    ])

    @app.callback([Output('moleculeviewer', 'children')],
                  [Input('dropdown', 'value')])
    def update_molecule3d(mode):
        '''
        Call the dashbio.molecule3dviewer and updated the coloring based on user input.
        '''
        # Calculate styles based on enrichment scores for the 3d viewer
        styles_data = _parse_styles_data(
            model_data, self.dataframe.copy(), temp_kwargs['gof'],
            temp_kwargs['lof'], mode, position_correction, chain
        )
        print (mode)


        return [html.Div(dashbio.Molecule3dViewer( # 3d molecule
                            modelData=model_data,
                            styles=styles_data,
                            selectionType='Chain'
                        ))]

    @app.callback([
        Output('heatmap', 'figure'),
        Output('mean', 'figure'),
    ], [
        Input('dropdown', 'value'),
    ])
    def update_graphs(mode='mean'):
        '''
        Aux function to update the plotly figures based on the user input.
        '''

        heatmap = self.heatmap_plotly(
            return_plot_object=True,
            show=False,
            title='Heatmap',
            figsize=(8, 2.5),
        )
        mean = self.mean_plotly(
            mode=mode,
            return_plot_object=True,
            show=False,
            title='Enrichment scores filtered by {}'.format(mode),
            figsize=(6, 2.5),
        )

        return heatmap, mean,

    # Run server
    app.run_server(port=8080, )

In [ ]:
pdb = '../../data/5p21.pdb'
dashboard_3d_protein(hras_RBD, pdb)

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is run

 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Dec/2020 21:28:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 21:28:04] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 21:28:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 21:28:04] "GET /_favicon.ico?v=1.17.0 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 21:28:05] "POST /_dash-update-component HTTP/1.1" 200 -


mean


127.0.0.1 - - [06/Dec/2020 21:28:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 21:28:09] "POST /_dash-update-component HTTP/1.1" 200 -


A


127.0.0.1 - - [06/Dec/2020 21:28:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 21:30:20] "POST /_dash-update-component HTTP/1.1" 200 -


N


127.0.0.1 - - [06/Dec/2020 21:30:20] "POST /_dash-update-component HTTP/1.1" 200 -


In [5]:
def _parse_pdb(pdb):
    '''return the pdb in jason format'''
    # Parse pdb file
    modata = parser.create_data(pdb)

    # Put in jason format
    fmodel = files_data_style(modata)
    with open(fmodel) as fm:
        model_data = json.load(fm)

    return model_data


def _parse_styles_data(
    model_data, df, gof, lof, mode, position_correction, chain
):
    '''
    From a dataframe with enrichment scores, this function will return a jason file
    with the color of each atom.
    
    Returns
    -------
    styles_data : jason file
    '''

    # Create empty dict
    styles_data = {}

    # Calculate df with colors
    df_color = _add_color(
        hras_RBD.dataframe.copy(), 0.1, -0.3, chain, position_correction=0
    )

    # Iterate over parsed pdb
    for item in model_data['atoms']:
        #if item['chain'] != 'A':  # only color atoms from selected chain
        #break
        try:
            style_atom = {
                'color': _assign_color_jason(df_color, item['residue_index']),
                'visualization_type': 'cartoon'
            }
            styles_data[str(item['serial'])] = style_atom
        except IndexError:  # in case we runt out of index
            pass
    return styles_data


def _assign_color_jason(df, residue):
    '''
    Give a color to the atom based on the enrichment score of that residue.
    As an input takes the dataframe that _add_color returns.
    '''
    return df.loc[df['Position_Corrected'] == residue, 'Color'].iloc[0]


def _add_color(df, gof, lof, mode, position_correction):
    '''You input the dataframe. Removes stop codons. 
    Returns the positions that are going to be colored blue,red and white'''

    # Correct position
    df['Position_Corrected'] = df['Position'] + position_correction

    # Add dummy color column
    red = '#FD3216'
    blue = '#6A76FC'
    green = '#16FF32'
    df['Color'] = green

    # Select grouping
    if mode == 'mean':
        df_grouped = df.groupby(['Position'], as_index=False).mean()
    else:
        df_grouped = df.loc[df['Aminoacid'] == mode]

    # Color of mutations
    df_grouped.loc[df_grouped['Score'] < lof, 'Color'] = blue
    df_grouped.loc[df_grouped['Score'] > gof, 'Color'] = red

    return df_grouped


def files_data_style(content):
    '''
    Function to create the modelData and style files for molecule visualization
    '''
    fdat = tempfile.NamedTemporaryFile(suffix=".js", delete=False, mode='w+')
    fdat.write(content)
    dataFile = fdat.name
    fdat.close()
    return dataFile

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df_bar = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges",
              "Bananas"], "Amount": [4, 1, 2, 2, 4, 5], "City":
    ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df_bar, x="Fruit", y="Amount", color="City", barmode="group")

app.layout = html.Div(
    children=[
        # All elements from the top of the page
        html.Div([
            html.H1(children='Hello Dash'),
            html.Div(
                children='''
            Dash: A web application framework for Python.
        '''
            ),
            dcc.Graph(id='graph1', figure=fig),
        ]),
        # New Div for all elements in the new 'row' of the page
        html.Div([
            html.H1(children='Hello Dash'),
            html.Div(
                children='''
            Dash: A web application framework for Python.
        '''
            ),
            dcc.Graph(id='graph2', figure=fig),
        ]),
    ]
)

if __name__ == '__main__':
    app.run_server(port=8069)

before bootstrap

In [ ]:
def dashboard_3d_protein(
    self, pdb, mode='mean', chain='A', position_correction=0, **kwargs
):
    '''
    Docstring
    '''

    # update kwargs
    temp_kwargs = copy.deepcopy(code_kwargs.kwargs())
    temp_kwargs.update(kwargs)

    # Load data from pdb file
    model_data = _parse_pdb(pdb)

    # Calculate styles based on enrichment scores
    styles_data = _parse_styles_data(
        model_data, self.dataframe.copy(), temp_kwargs['gof'],
        temp_kwargs['lof'], mode, position_correction, chain
    )

    # Run dashhoard
    _run_dashboard_3d_protein(self, model_data, styles_data)


def _run_dashboard_3d_protein(self, model_data, styles_data):
    '''
    Create dashboard for main function.
    '''
    # Style sheet
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

    # Open app
    app = dash.Dash('__main__', external_stylesheets=external_stylesheets)

    # App layout
    app.layout = html.Div(
        id="app-container",
        children=[
            # first row
            html.Div([
                    dcc.Graph( # For plotly graphs I need to use dcc
                        id="further right-column",
                        #className="six columns",
                        figure=self.mean_plotly(
                            return_plot_object=True, show=False,
                            title='',
                            figsize=(8,2.5),
                        )
                    )
            ]),
            # second row
            html.Div([
                # Left column
                html.Div(
                    id="left-column",
                    className="six columns",
                    children=[
                        dashbio.Molecule3dViewer(
                            id='my-dashbio-molecule3d',
                            modelData=model_data,
                            styles=styles_data,
                            selectionType='Chain'
                        )
                    ]
                ),
                # Right columns
                html.Div(
                    [html.Div(
                        dcc.Graph( # For plotly graphs I need to use dcc
                            id="right-column",
                            #className="six columns",
                            figure=self.histogram_plotly(
                                return_plot_object=True, show=False,
                                title='',
                                figsize=(4,2.5),

                            )
                        ),

                    ),
                ],
                className="six columns")
            ])
        ]
    )

    # Run server
    app.run_server(port=8071, )

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

import pandas as pd

df = pd.read_csv(
    'https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv'
)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider', figure={}),
    dcc.Slider(
        id='year-slider',
        min=df['year'].min(),
        max=df['year'].max(),
        value=df['year'].min(),
        marks={str(year): str(year) for year in df['year'].unique()},
        step=None
    )
])


@app.callback(
    Output('graph-with-slider', 'figure'),
)
def update_figure():
    filtered_df = df[df.year == 1992]

    fig = px.scatter(
        filtered_df,
        x="gdpPercap",
        y="lifeExp",
        size="pop",
        color="continent",
        hover_name="country",
        log_x=True,
        size_max=55
    )

    fig.update_layout(transition_duration=500)

    return fig


if __name__ == '__main__':
    app.run_server(port=8093)